In [1]:
import csv
import json
import datetime
separator = "\t" #Définir votre propre séparateur Ex: '\t', ' '

from collections import defaultdict
def timedelta_def(): return datetime.timedelta()
def returnnone(): return None
def defaultdicttimedalta(): return defaultdict(timedelta_def)
def defaultdictseption(): return defaultdict(defaultdicttimedalta)
maxdict = lambda dict: max(dict, key=lambda key: dict[key])

#/\/\/\/\/\/\ Nom de la métrique: Extraction des Points d’Intérêts /\/\/\/\/\/\
#Le but de cette métrique est de détecter les points d'intérêts d'un individu. 
#Les points d'intérêts correspondent aux lieux où l'utilisateur a le plus séjourné.
#
#Dans le cadre de ce fichier d'utilité, nous regardons par défaut 3 POI les plus importants pour les 12 semaines du dataset.
#1 POI correspond à trois éléments: Lieu d'habitation (22h à 6h), lieu de travail (9h à 16h) et lieu d'activité (le weekend de 10h à 18h) 
#
#L'idée globale de cette utilité est de s'assurer que l'on retrouve bien dans le fichier anonymisé les lieux clé de la vie d'un individu.
#
#Le score est calculé de la manière suivante :
#	Somme [pour chaque i individu] : 
#		Somme [pour chaque POI] :
#			Si temps_POI_oriFile > temps_POI_anonymFile :
#				temps_POI_anonymFile / temps_POI_oriFile
#			Sinon :
#				temps_POI_oriFile / temps_POI_anonymFile
#Le score est finalement converti sur 1
#/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\

#######################################
# --- Taille des points d'intérêts ---#
#######################################
size = 2
# 4 : cellule au mètre
# 3 : cellule à la rue
# 2 : cellule au quartier
# 1 : cellule à la ville
# 0 : cellule à la région
# -1 : cellule au pays

######################################
# --- Nb de POI à vérifier par ID ---#
######################################
nbPOI = 3
# 3: Vérification des 3 POI les plus fréquentés en terme de temps de présence.

################################
# --- Définition des heures ---#
################################
# Détection des POI -nuit, travail et weekend- durant les heures suivantes:
night_start, night_end = 22, 6
# De 22h00 à 6h00
work_start, work_end = 9, 16
# Lunch time
lunch_time = [12,13]
# De 9h00 à 16h00
weekend_start, weekend_end = 10, 18

In [6]:
def getMaxElement(theDict):
	result = defaultdict(timedelta_def)
	for _ in range(nbPOI):
		if len(theDict)==0:
			break
		key = maxdict(theDict)
		result[key] = theDict[key]
		del theDict[key]
	return result


In [7]:
last_date_original_tab = defaultdict(returnnone)
last_date_anonymised_tab = defaultdict(returnnone)
def diff_time(key, time1, last_date_tab):
    if last_date_tab[key] is None:
        last_date_tab[key] = time1
        return datetime.timedelta()
    else:
        difference = time1 - last_date_tab[key]
        last_date_tab[key] = time1
        return difference
    

In [10]:

def main(originalFile, parameters=None):
	if parameters is None:
		parameters={"size":2,"nbPOI":1,"night_start":22,"night_end":6,"work_start":9,"work_end":16,"weekend_start":10,"weekend_end":18}
	global size
	size = parameters.get("size", 2)
	global nbPOI
	nbPOI = parameters.get("nbPOI", 1)
	global night_start
	night_start = parameters.get("night_start", 22)
	global night_end
	night_end = parameters.get("night_end", 6)
	global work_start
	work_start = parameters.get("work_start", 9)
	global work_end
	work_end = parameters.get("work_end", 16)
	global weekend_start
	weekend_start = parameters.get("weekend_start", 10)
	global weekend_end
	weekend_end = parameters.get("weekend_end", 18)
	
	fd_original = open(originalFile, newline='')
	# fd_anonymised = open(anonymisedFile, newline='')
	original_reader = csv.reader(fd_original, delimiter=separator)
	# anonymised_reader = csv.reader(fd_anonymised, delimiter=separator)
	
	tabOri = defaultdict(defaultdictseption)
	# tabAno = defaultdict(defaultdictseption)

	for lineOri in original_reader:
	
		#--- Original file
		id = lineOri[0]
		date_time = datetime.datetime.fromisoformat(lineOri[1][:19])
		calendar = date_time.date().isocalendar()
		key = (id)
		#key = (id, calendar[0], calendar[1])
		
		gps = (round(float(lineOri[2]),size), round(float(lineOri[3]),size)) 
		
		if date_time.weekday()<5:
			if date_time.time()>datetime.time(night_start,00) or date_time.time()<datetime.time(night_end,00):
				tabOri[key]['night'][gps] += diff_time(key, date_time, last_date_original_tab)
			elif date_time.time()>datetime.time(work_start,00) and date_time.time()<datetime.time(work_end,00) and date_time.time().hour not in lunch_time:
				tabOri[key]['work'][gps] += diff_time(key, date_time, last_date_original_tab)
				print(date_time)
		# else:
		# 	if date_time.time()>datetime.time(weekend_start,00) and date_time.time()<datetime.time(weekend_end,00):
		# 		tabOri[key]['weekend'][gps] += diff_time(key, date_time, last_date_original_tab)
		print(f"ID: {id}, Date: {date_time}, GPS: {gps}")
		print()

	for user_id, poi_data in tabOri.items():
		print(f"User ID: {user_id}")
		for poi_type, gps_data in poi_data.items():
			print(f"  POI Type: {poi_type}")
			for gps, duration in gps_data.items():
				print(f"    Location: {gps}, Time Spent: {duration}")



In [12]:
main ("/Users/quynhnguyen/Documents/Documents/Project Data/Anonymisation Project/Anonymisation-Project/geo_data_generator/origin_data/survey_results_1.csv")

2025-01-01 09:22:26
ID: 0, Date: 2025-01-01 09:22:26, GPS: (48.81, 2.23)

2025-01-01 15:00:44
ID: 4, Date: 2025-01-01 15:00:44, GPS: (48.91, 2.3)

ID: 4, Date: 2025-01-01 03:07:59, GPS: (48.94, 2.43)

ID: 5, Date: 2025-01-01 21:27:55, GPS: (48.85, 2.39)

ID: 5, Date: 2025-01-01 20:15:57, GPS: (48.85, 2.39)

ID: 6, Date: 2025-01-01 21:55:24, GPS: (48.82, 2.32)

ID: 6, Date: 2025-01-01 12:07:10, GPS: (48.91, 2.3)

ID: 7, Date: 2025-01-01 08:48:24, GPS: (48.9, 2.26)

2025-01-01 11:11:32
ID: 7, Date: 2025-01-01 11:11:32, GPS: (48.9, 2.26)

ID: 8, Date: 2025-01-01 03:02:38, GPS: (48.94, 2.24)

2025-01-02 14:56:39
ID: 1, Date: 2025-01-02 14:56:39, GPS: (48.84, 2.43)

2025-01-02 15:43:29
ID: 1, Date: 2025-01-02 15:43:29, GPS: (48.85, 2.39)

ID: 3, Date: 2025-01-02 08:21:26, GPS: (48.85, 2.29)

2025-01-02 14:11:10
ID: 3, Date: 2025-01-02 14:11:10, GPS: (48.85, 2.29)

ID: 4, Date: 2025-01-02 19:28:04, GPS: (48.94, 2.44)

ID: 4, Date: 2025-01-02 21:18:35, GPS: (48.94, 2.43)

ID: 5, Date: 2025-01